<a href="https://colab.research.google.com/github/youngchanseo/Hon.Gong.Machine-SNU/blob/main/(05_3_3)_Histogram_based_Gradient_Boosting_p272.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Full code: https;//bit.ly/hg-05-3
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://raw.githubusercontent.com/youngchanseo/Hon.Gong.Machine-SNU/main/(05_3_0)%20Raw%20data.csv')# https://bit.ly/wine_csv_data
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

#Histogram-based Gradient Boosting
정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높음
먼저 입력 특성을 256개 구간으로 나눔 -> 노드를 분할할 때 최적의 분할을 매우 빠르게
 Histogram based Gradient boosting은 256개 구간중에 하나를 떼어놓고, 누락된 값을 위해 사용됨

 HistGradientBosstingClassifier
 - 트리의 개수를 지정하는데 max_iter (부스팅 반복횟수 지정)을 사용


In [7]:
from sklearn.model_selection import cross_validate
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier (random_state =42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대 적합을 잘 억제하면서, Gradient Boosting 보다 조금 더 높은 성능을 제공함

특성 중요도 확인:
- permutation_importance()함수: 하나씩 Random하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산

In [20]:
#histogram based graident boosting model을 훈련하고, 훈련세트에서 특성 중요도를 계산
# n_repeats 매개변수: 랜덤하게 섞을 회수 지정. 기본값 5

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance (hgb, train_input, train_target, n_repeats = 10, # n_repeats 매개변수: 랜덤하게 섞을 회수 지정. 기본값 5
                                random_state=42, n_jobs=-1)
print(result.importances_mean)


[0.08876275 0.23438522 0.08027708]


 permutation_importance () 함수가 반환하는 객체
- 반복하여 얻은 특성 중요도(importances)
- 평균 (importances_mean)
- 표준 편차 (importances_std)



In [21]:
#Test set

result = permutation_importance (hgb, test_input, test_target,
                                 n_repeats = 10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


Test set의 결과를 보면 ( [0.05969231 0.20238462 0.049])

Gradient boosting ([0.15872278 0.68010884 0.16116839])과 비슷하게 좀 더 당도에 집중

In [22]:
#test set 에서의 성능을 최종적으로 확인

hgb.score(test_input, test_target)

0.8723076923076923

Test set에서는 87 % 정확도

실전에 투입하면 성능은 이보다는 좀 더 낮을것임

In [24]:
# 가장 대표적인 library: XGBoost

from xgboost import XGBClassifier
xgb = XGBClassifier (tree_method='hist', random_state=42)
scores = cross_validate (xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [27]:
#Microsoft LightGBM

from lightgbm import LGBMClassifier
lgb = LGBMClassifier (random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
